## Example: Multiple Objective Optimization with Constraints
`ENTMOOT` also supports multi-objective optimization according to:

```
@article{thebelt2022multi,
  title={Multi-objective constrained optimization for energy applications via tree ensembles},
  author={Thebelt, Alexander and Tsay, Calvin and Lee, Robert M and Sudermann-Merx, Nathan and Walz, David and Tranter, Tom and Misener, Ruth},
  journal={Applied Energy},
  volume={306},
  pages={118061},
  year={2022},
  publisher={Elsevier}
}
```

An example that derives Pareto-optimal points of the
[Fonzeca Freming](https://en.wikipedia.org/wiki/Test_functions_for_optimization) is given in the
following:

Initialize the search space manually.

In [ ]:
from entmoot.benchmarks import FonzecaFleming
from entmoot.optimizer import Optimizer

# initialize multi-objective test function
funcMulti = FonzecaFleming()

# define optimizer object and specify num_obj=2
opt = Optimizer(funcMulti.get_bounds(),
                num_obj=2,
                n_initial_points=10,
                random_state=100)

# main BO loop that derives pareto-optimal points
for _ in range(50):
    next_x = opt.ask()
    next_y = funcMulti(next_x)
    opt.tell(next_x,next_y)

get the core of the gurobi model from helper function 'get_core_gurobi_model'

In [ ]:
from entmoot.optimizer.gurobi_utils import get_core_gurobi_model
core_model = get_core_gurobi_model(space)

ordering of variable indices is dependent on space definition

cont_var_dict contains all continuous variables

In [ ]:
x0 = core_model._cont_var_dict[0]
x1 = core_model._cont_var_dict[1]

cat_var_dict contains all categorical variables

In [ ]:
x2 = core_model._cat_var_dict[2]

define constraints accordingly

In [ ]:
core_model.addConstr(x0 + x1 >= 2)
core_model.addConstr(x1 == 1)
core_model.update()

Define Optimizer object

In [ ]:
opt = Optimizer(func.get_bounds(),
                base_estimator="ENTING",
                n_initial_points=0,
                initial_point_generator="random",
                acq_func="LCB",
                acq_optimizer="global",
                random_state=100,
                acq_func_kwargs=None,
                acq_optimizer_kwargs={
                    "add_model_core": core_model
                },
                model_queue_size=None,
                base_estimator_kwargs={
                    "lgbm_params": {"min_child_samples": 1}
                },
                verbose=True,
                )

add initial points that are feasible

In [ ]:
for x in [(1.1, 1.0, 'mult6'), (1.2, 1.0, 'mult6'), (1.3, 1.0, 'pow2')]:
    opt.tell(x, func(x))

run optimizer for 20 iterations

In [ ]:
res = opt.run(func, n_iter=20)
print(f"-> best solution found {res.fun}")